In [214]:
import pandas as pd
import numpy as np
import itertools

In [105]:
filename = "/Users/lmatayoshi/Documents/Projects/repo-ciencia-datos/tp-4/clean_data.csv"

In [161]:
def filter_word(word):
    word = word.replace(" ", "")
    word = word.replace("--", "-")
    word = word.upper()
    return word.replace("\\.", "")

In [197]:
co_autores = []
autores = []
lobos_solitarios = []

with open(filename,'r') as f:
    for line in f:
        line = line.split(',')
        line = line[3].split('&')
        cleaned_line = [filter_word(w) for w in line]
        if len(cleaned_line) > 1:
            co_autores.append(cleaned_line)
            for i in cleaned_line:
                autores.append(i)
        else:
            lobos_solitarios.append(cleaned_line)

autores= list(set(autores))

In [198]:
 #6,7, 12
black_list = [
    ')',
    '-p-dedonder',
    '10',
    '14p',
    'a',
    'pages',
    'p',
    'eds.h.-d.doebnere.a.worldsci.',
    'conclusions'
]


In [190]:
for b in black_list:
    print b
    autores.remove(b.upper())

)
-p-dedonder
10
14p
a
pages
p
eds.h.-d.doebnere.a.worldsci.
conclusions


In [242]:
autores_size = len(autores)
print autores_size
#co_autores
adys = np.zeros((autores_size,autores_size))

3684


In [243]:
co_autores_data = pd.DataFrame(data=adys, columns=autores, index=autores)

In [244]:
#prueba =co_autores[0:5]
#adys = np.zeros((5,5))
#co_autores_prueba = pd.DataFrame(data=adys, columns=autores[0:5], index=autores[0:5])

In [249]:
for TUPLA in co_autores:
    pares=list(itertools.combinations(TUPLA, 2))
    for i in pares:
        co_autores_data[i[0]][i[1]]=co_autores_data[i[0]][i[1]]+1
        co_autores_data[i[1]][i[0]]=co_autores_data[i[1]][i[0]]+1

In [250]:
co_autores_data

,AOKI,RIETDIJK,SPECTOR,LAZKOZ,RUSAKOV,RAJEEV,HAUER,MARSHAKOV,PORRATI,OOTA,...,MCAVITY,GRIGORIYN,ERDMENGER,MEURICE,JAHN,NIERSTE,TUTUTI,LESGOURGUES,HAUSSLING,YORK
AOKI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RIETDIJK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SPECTOR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LAZKOZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RUSAKOV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RAJEEV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HAUER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MARSHAKOV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PORRATI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OOTA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
